In [1]:
import pandas as pd
from util import read_data, save, plt_encoding_error, error, normal
import matplotlib.pyplot as plt

In [2]:
train, test_A, _ = read_data()

num_train = len(train)
num_test_A = len(test_A)

train_m = pd.concat([train, test_A])

In [3]:
train_m['甘油三酯_normal'] = train_m['甘油三酯'].apply(normal, args=(1.69,))
train_m['尿素_normal'] = train_m['尿素'].apply(normal, args=(7.1,))
train_m['尿酸_normal'] = train_m.apply(lambda x: 0 if (x['性别']=='女' and x['尿酸']>357) or (x['性别']=='男' and x['尿酸']>416) else 1, axis=1)
train_m['*天门冬氨酸氨基转换酶_normal'] = train_m['*天门冬氨酸氨基转换酶'].apply(normal, args=(40,))

In [ ]:
#添加统计量
#columns = total.columns.tolist()
columns = ["年龄","甘油三酯","红细胞平均体积","尿素","尿酸","*碱性磷酸酶","红细胞平均血红蛋白浓度","*天门冬氨酸氨基转换酶",
          "*r-谷氨酰基转换酶",
          ]

for column in columns:
    if column != "体检日期" and column != "性别":
        max_value = train_m[column].max()
        min_value = train_m[column].min()
        avg_value = train_m[column].mean()
        train_m[column+"max"] = train_m[column].apply(lambda x : x - max_value)
        #total[column+"min"] = total[column].apply(lambda x : x - min_value)
        #total[column+"avg"] = total[column].apply(lambda x : x - avg_value)

In [ ]:
# 年龄分组
# age_cut = pd.cut(train_m['年龄'],[1,20,30,35,40,43,46,49,52,55,58,61,64,67,70,73,76,79,82,85,88])
# age_cut = pd.DataFrame({'age':age_cut})
# #age_cut.rename(columns={'年龄': 'age'}, inplace=True)
# train_m = pd.concat([train_m, age_cut], axis=1)
# train_m.drop(['年龄'], axis=1, inplace=True)

In [4]:
drop_list = ['id', '血糖','体检日期','性别', '血小板比积', '血小板计数', '嗜酸细胞%']

In [5]:
train_y = train['血糖']
train_m.drop(drop_list, axis=1, inplace=True)

In [ ]:
# train_m = pd.get_dummies(train_m, columns=['age'])

In [6]:
# 重新切分训练与测试数据
train_x = train_m.iloc[:num_train]
test_A_new = train_m.iloc[num_train:num_test_A + num_train]

In [7]:
features = train_x.columns.tolist()
train_X = train_x.as_matrix()
train_Y = train_y.as_matrix()

test_X = test_A_new.as_matrix()

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.2, random_state=1)

In [ ]:
# from sklearn.metrics import make_scorer
# def my_mes(estimator, X_test, y_test):
#     y_pred = estimator.predict(X_test)
#     return (mean_squared_error(y_test, y_pred) * 0.5)
# make_scorer(my_mes, greater_is_better=False)

In [10]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

param_grid = dict(num_leaves=[31,48],
                  n_estimators=[250, 200],
                  learning_rate=[0.01, 0.02,0.03],
                  min_child_samples=[10, 20, 40],
                  subsample=[0.8, 0.9])
lgb_regressor = lgb.LGBMRegressor(
                  objective='regression',
#                   min_child_samples=20,
                  subsample_freq=1,
                  colsample_bytree=1.0,
                  reg_alpha=1.0,
                  reg_lambda=0.0,
                  n_jobs=-1)
grid = GridSearchCV(cv=5, estimator=lgb_regressor, n_jobs=4, param_grid=param_grid, scoring='neg_mean_squared_error')
grid.fit(train_X, train_Y)
print('Best parameters found by grid search are:', grid.best_params_, '  best_score: ', grid.best_score_)

C:\Users\kyle\Anaconda3\lib\site-packages\lightgbm\basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
C:\Users\kyle\Anaconda3\lib\site-packages\lightgbm\basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


Best parameters found by grid search are: {'learning_rate': 0.02, 'min_child_samples': 20, 'n_estimators': 200, 'num_leaves': 31, 'subsample': 0.8}   best_score:  -1.97123751136


In [ ]:
# result = grid.predict(test_X)
# data1 = pd.DataFrame(result)
# save(data1, 'lgb_grid_cv')

In [20]:
params = {
    'boosting': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'min_data_in_leaf': 20,
    'learning_rate': 0.02,
    'lambda_l1':1,
#     'lambda_l2':1,
    'cat_smooth':10,
    'feature_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [21]:
from sklearn.model_selection import KFold
import lightgbm as lgb
def cv_estimate(n_splits, lgb, train_X, train_Y):
    cv = KFold(n_splits=n_splits)
    val_scores = 0
    for train, test in cv.split(train_X, train_Y):
        train_x_k, train_y_k= train_X[train], train_Y[train]                
        test_x_k, test_y_k = train_X[test], train_Y[test]
        
        lgb_train = lgb.Dataset(train_x_k, train_y_k, feature_name = features)
        lgb_test = lgb.Dataset(test_x_k, test_y_k)
         
        gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=200)
        y = gbm.predict(test_x_k, num_iteration=gbm.best_iteration)
        val_scores += error(y, test_y_k)
    val_scores /= n_splits
    return val_scores

scores = cv_estimate(5, lgb, train_X, train_Y)
print(scores)

1/2 Mean squared error: 0.605948
1/2 Mean squared error: 0.963798
1/2 Mean squared error: 0.948488
1/2 Mean squared error: 1.421974
1/2 Mean squared error: 1.005316
0.989105001719


In [22]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train, feature_name = features)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, feature_name = features)

train_all = lgb.Dataset(train_X, train_Y, feature_name = features)

# specify your configurations as a dict

print('Start training...')
# train
gbm = lgb.train(params,
                   lgb_train,
                    num_boost_round=1000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=20)

print('Start predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
error(y_test, y_pred)

# online

predict = gbm.predict(test_X, num_iteration=gbm.best_iteration)
data1 = pd.DataFrame(predict)
# save
save(data1, 'lgb')

# gbm_online = lgb.train(params,
#                 train_all,
#                 num_boost_round=280)
# # predict
# predict = gbm_online.predict(test_X, num_iteration=gbm_online.best_iteration)
# data1 = pd.DataFrame(predict)
# # save
# save(data1, 'lgb')

Start training...
[1]	valid_0's l2: 2.09765
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's l2: 2.08529
[3]	valid_0's l2: 2.07449
[4]	valid_0's l2: 2.06443
[5]	valid_0's l2: 2.05533
[6]	valid_0's l2: 2.04363
[7]	valid_0's l2: 2.03635
[8]	valid_0's l2: 2.02655
[9]	valid_0's l2: 2.0186
[10]	valid_0's l2: 2.01077
[11]	valid_0's l2: 2.0034
[12]	valid_0's l2: 1.99479
[13]	valid_0's l2: 1.98459
[14]	valid_0's l2: 1.97565
[15]	valid_0's l2: 1.96973
[16]	valid_0's l2: 1.96318
[17]	valid_0's l2: 1.95758
[18]	valid_0's l2: 1.94976
[19]	valid_0's l2: 1.94448
[20]	valid_0's l2: 1.93673
[21]	valid_0's l2: 1.92974
[22]	valid_0's l2: 1.92633
[23]	valid_0's l2: 1.92022
[24]	valid_0's l2: 1.91578
[25]	valid_0's l2: 1.91048
[26]	valid_0's l2: 1.90433
[27]	valid_0's l2: 1.90055
[28]	valid_0's l2: 1.89504
[29]	valid_0's l2: 1.89306
[30]	valid_0's l2: 1.88747
[31]	valid_0's l2: 1.88208
[32]	valid_0's l2: 1.87582
[33]	valid_0's l2: 1.8717
[34]	valid_0's l2: 1.86697
[35]	valid_0's

In [ ]:
plt_encoding_error()

In [ ]:
lgb.plot_importance(gbm)
plt.show()